In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import PIL
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#출력되는 내용은 (세로, 가로, 채널 수)

with open('/content/gdrive/MyDrive/캡스톤/조양방직1.jpg') as file_object:
    image1 = cv2.imread('/content/gdrive/MyDrive/캡스톤/조양방직1.jpg')
    print(image.shape)

with open('/content/gdrive/MyDrive/캡스톤/조양방직2.jpg') as file_object:
    image2 = cv2.imread('/content/gdrive/MyDrive/캡스톤/조양방직2.jpg')
    print(image2.shape)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


AttributeError: ignored

In [ ]:
#사진 크기 x값 y값 0.5배
#img = cv2.resize(image1, dsize=(0,0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
#사진 해상도 강제로 480x640 고정
img_size = (480, 640)

img1 = cv2.resize(image1, dsize=img_size)
img2 = cv2.resize(image2, dsize=img_size)

cv2_imshow(img1)
cv2_imshow(img2)

In [ ]:
def show_img_compare(img,img1):
  f, ax = plt.subplots(1, 2, figsize=(10,10))
  ax[0].imshow(img)
  ax[1].imshow(img1)
  ax[0].axis('off')
  ax[1].axis('off')
  f.tight_layout()
  plt.show()

In [ ]:
show_img_compare(img1, img2)

In [ ]:
#평균 픽셀값 비교
img_temp1 = img1.copy()
img_temp1[:,:,0], img_temp1[:,:,1], img_temp1[:,:,2] = np.average(img1, axis=(0,1))

img_temp2 = img2.copy()
img_temp2[:,:,0], img_temp2[:,:,1], img_temp2[:,:,2] = np.average(img2, axis=(0,1))

show_img_compare(img1, img_temp1)
show_img_compare(img2, img_temp2)

In [ ]:
#가장 많이 해당되는 픽셀 색깔 출력

print(img1.reshape(-1, 3).shape)
img1.reshape(-1,3)

In [ ]:
unique, counts = np.unique(img1.reshape(-1, 3), axis=0, return_counts=True)
print(unique)
print(counts)

In [ ]:
#실패
img_temp1 = img1.copy()
unique, counts = np.unique(img_temp1.reshape(-1, 3), axis = 0, return_counts=True)
img_temp1[:,:,0], img_temp1[:,:,1], img_temp1[:,:,2] = unique[np.argmax(counts)]

img_temp2 = img2.copy()
unique, counts = np.unique(img_temp2.reshape(-1, 3), axis = 0, return_counts=True)
img_temp2[:,:,0], img_temp2[:,:,1], img_temp2[:,:,2] = unique[np.argmax(counts)]

show_img_compare(img1, img_temp1)
show_img_compare(img2, img_temp2)

In [ ]:
from sklearn.cluster import KMeans
#많이 출력되는 색깔 TOP5
#n_clusters 갯수 = 추출할 색깔 갯수
clt = KMeans(n_clusters = 5)  

def palette(clusters):
  width = 300
  palette = np.zeros((50, width, 3), np.uint8)
  steps = width/clusters.cluster_centers_.shape[0]
  for idx, centers in enumerate(clusters.cluster_centers_):
    palette[:, int(idx*steps):(int((idx+1)*steps)), :] = centers
  return palette

new_img1 = clt.fit(img1.reshape(-1,3))
show_img_compare(img1, palette(new_img1))

new_img2 = clt.fit(img2.reshape(-1,3))
show_img_compare(img2, palette(new_img1))

In [ ]:
from collections import Counter
#많이 출력되는 색깔 TOP5(빈도수도 고려)

def color(k_cluster):
  width = 300
  palette = np.zeros((50, width, 3), np.uint8)

  n_pixels = len(k_cluster.labels_)
  counter = Counter(k_cluster.labels_)
  perc = {}
  for i in counter:
    perc[i] = np.round(counter[i]/n_pixels, 2)
  perc = dict(sorted(perc.items()))

  print(perc)
  print(k_cluster.cluster_centers_)

  step = 0

  for idx, centers in enumerate(k_cluster.cluster_centers_):
    palette[:, step:int(step + perc[idx]*width+1), :] = centers
    step += int(perc[idx]*width+1)

  return palette

new_img1 = clt.fit(img1.reshape(-1,3))
show_img_compare(img1, color(new_img1))

new_img2 = clt.fit(img2.reshape(-1,3))
show_img_compare(img2, color(new_img2))